# Aviation Costing

<center>
    <img src ='https://i.imgur.com/J2L96N4.png' alt= 'Airlines Logos'>
</center>

*The aviation industry operates as a complex economic ecosystem, involving intricate cost structures that span aircraft manufacturing, operational expenses, infrastructure maintenance, and regulatory compliance. The industry's financial landscape is shaped by factors such as fuel prices, labor costs, technological investments, and the cyclical nature of demand. Balancing competitive pricing for air travel with the need for profitability presents an ongoing challenge, further compounded by the expenses associated with safety measures and environmental initiatives. Effective cost management is crucial for airlines, manufacturers, and service providers to ensure sustainable growth while adapting to evolving economic and market conditions.*

## Challenge:

" *If the payload is discounted of the luggage weight, what will be the reduction in operating cost during the flight? Find the costing factor by which the fuel cost will drop.* "

## Solution:

<center>
    <img src='https://i.imgur.com/M7vyVLm.png' alt= 'Website Calculator'>
</center>

This is a glimpse of the website which we will develop and deploy in this notebook for calculating costing factor and analyzing other key factors.

### Breguet's Range Equation

The Breguet Range Equation is a fundamental mathematical model used in aviation engineering to estimate the maximum range an aircraft can achieve based on its aerodynamic efficiency and fuel consumption characteristics.

$$\boxed{R = \frac{V}{C} \cdot \frac{1}{L/D} \cdot ln\left(\frac{W_{intital}}{W_{final}} \right)}$$

where:
- $R$: range of the aircraft (in NM)
- $V$: true air speed of the aircraft (in kts)
- $C$: fuel consumption rate  (fuel flow rate) of the aircraft (generally in lb/hr, here in lb/NM)
- $L/D$: lift-to-drag ratio of the aircraft
- $W_{initial}$: initial weight of the aircraft, including fuel and payload.
- $W_{final}$: final weight of the aircraft, which is the sum of the empty weight and payload weight after fuel consumption.

#### Helping formulae for derivation:

- $fuel\ consumption\ rate= \frac{fuel\ flow}{time}$
- $endurance=\frac{fuel\ capacity}{fuel\ flow}$
- $range= endurance \cdot true\ air\ speed$
- $fuel\ consumed= fuel\ flow \cdot time$

Using these formulae, we can derive that

$$\boxed{fuel\ capacity = \frac{fuel\ consumed}{C} \cdot ln\left(\frac{W_1}{W_2}\right) \cdot \frac{1}{L/D}}$$

### Deriving the Costing Factor

Here, let us consider two conditions:
- maximum payload condition
- test payload condition

Here, we are assuming that the same aircraft is put under the above two conditions, therefore, the fuel capacity for the aircraft will be same.

Let us represent the test payload condition variables by a naught ( $^o$ ).

$\frac{fuel\ consumed}{C} \cdot ln\left(\frac{W_1}{W_2}\right) \cdot \frac{1}{L/D} = \frac{fuel\ consumed^o}{C^o} \cdot ln\left(\frac{{W^o}_1}{{W^o}_2}\right) \cdot \frac{1}{L^o/D^o}$

on simplifying the above equation we will obtain,

$$\boxed{fuel\ consumed^o = \frac{C^o}{C}\cdot \frac{L^o/D^o}{L/D}\cdot\frac{{ln\left(\frac{W_1}{W_2}\right)}}{{ln\left(\frac{{W_1}^o}
{{W_2}^o}\right)}} \cdot fuel\ consumed}$$ 

we can replace the factor by $\gamma$, and we will get

$$\boxed{fuel\ consumed^o = \gamma \cdot fuel\ consumed} : 0<\gamma<1$$

From here, we can observe that we have derived a factor which is a function of:
- fuel consumption rate
- lift-to-drag ratio
- initial weight
- final weight

#### Profit per NM:

Additionally, let us derive one more factor, which is the profit made by airlines per nautical mile.

$total\ cost = cost\ per\ pound \cdot fuel\ consumed$

$cost\ difference = cost\ per\ pound \cdot(fuel\ consumed - fuel\ consumed^o)$

$$\boxed{profit\ per\ NM = \frac{cost\ difference}{distance\ covered}}$$

One more concept which will come in handy is that final weight can be calulated from the initial weight. But before finding that, we have to find a simplified formula for calculating the initial weight of the aircraft.

$\therefore$ **initial weight = basic empty weight + fuel weight + crew weight + total seats filled x avg passenger weight + luggage weight + other variable weight**

and, **final weight = initial weight - fuel consumed x distance covered**

Now that we have all the mathematical tools and equations that we needed to find the costing factor, let us transform the same in form of a python program and deploy it to streamlit web-server.

## Program

These are the only libraries that we need for this website.

In [ ]:
import streamlit as st
import math
import matplotlib.pyplot as plt

First, let us design the sidebar for the website.

In [ ]:
# code for sidebar
with st.sidebar:
    # aircraft specifications 
    st.header('Aircraft Specifications')
    basic_empty_weight       = st.number_input('Basic Empty Weight of Aircraft(in lbs)', value = 0)
    number_of_seats          = st.number_input('Total Number of Passenger Seats in Aircraft', value = 0)
    maximum_payload_capacity = st.number_input('Maximum Payload Capacity of Aircraft(in lbs)', value = 0)

    # flight details
    st.header('Flight Details')
    fuel_weight             = st.number_input('Fuel Weight of Aircraft(in lbs)', value = 0)
    crew_weight             = st.number_input('Total Crew Weight in Aircraft(in lbs)', value = 0)
    other_variable_weight   = st.number_input('Other Variable Item Weight in Aircraft(in lbs)', value = 0)

    # max payload condition
    st.header('Max Payload Condition')
    max_payload_total_fuel_consumed     = st.number_input('Fuel Consumed by Aircraft under Max. Payload(in lbs)', value = 0)
    max_payload_fuel_consumption_rate   = st.number_input('Max. Payload Fuel Consumption of Aircraft(in lbs/NM)', value = 0.0)
    max_payload_lift_drag_ratio         = st.number_input('Max. Payload Lift-Drag Ratio of Aircraft', value = 0.0)

    # test payload condition
    st.header('Test Payload Condition')
    test_payload_fuel_consumption_rate  = st.number_input('Test Payload Fuel Consumption of Aircraft(in lbs/NM)', value = 0.0)
    test_payload_lift_drag_ratio        = st.number_input('Test Payload Lift-Drag Ratio of Aircraft', value = 0.0)

Now, let us design the main panel.

In [ ]:
st.header('Aviation Costing')
st.write('##### Test Case Inputs:')
# code for main panel
distance_col, avg_passenger_weight_cols, total_seats_filled_col, luggage_weight_col = st.columns(4)
with distance_col:
    distance             = st.number_input('Total Flight Distance(in NM)', value = 0)

with avg_passenger_weight_cols:
    avg_passenger_weight = st.number_input('Avg. Passenger Wt.(in lbs)', value = 180)

with total_seats_filled_col:
    total_seats_filled   = st.number_input('Total Number of Filled Seats', value = 0)
    
with luggage_weight_col:
    luggage_weight       = st.number_input('Total Luggage Weight(in lbs)', value = 0)

Now, let us use the initial and final weight equations for calculation. 

In [ ]:
# calculating initial and final weight
# maximum payload condition
max_payload_initial_weight  = basic_empty_weight + fuel_weight + crew_weight + number_of_seats*avg_passenger_weight +  maximum_payload_capacity + other_variable_weight
max_payload_final_weight    = max_payload_initial_weight - max_payload_fuel_consumption_rate*distance
# test payload condition
test_payload_initial_weight = basic_empty_weight + fuel_weight + crew_weight + total_seats_filled*avg_passenger_weight + luggage_weight + other_variable_weight
test_payload_final_weight   = test_payload_initial_weight - test_payload_fuel_consumption_rate*distance

Displaying the calculated weights in 2x2 orientation.

In [ ]:
# show initial and final weight
row1_col1, row1_col2 = st.columns(2)
row2_col1, row2_col2 = st.columns(2)

# maximum payload condition
with row1_col1:
    st.write(f'Full Payload Initial Weight: {max_payload_initial_weight}')

with row1_col2:
    if max_payload_final_weight >= 0:
        st.write(f'Full Payload Final Weight: {max_payload_final_weight}')
    else:
        st.write('Full Payload Final Weight: invalid inputs')

# test payload condition
with row2_col1:
    st.write(f'Test Payload Initial Weight: {test_payload_initial_weight}')

with row2_col2:
    if test_payload_final_weight >= 0:
        st.write(f'Test Payload Final Weight: {test_payload_final_weight}')
    else:
        st.write('Test Payload Final Weight: invalid inputs')

st.markdown('----')

Calculating fuel consumed under test payload condition.

In [ ]:
# calculating fuel consumed under test payload condition
test_payload_total_fuel_consumed = 0.0  # Default value

if (max_payload_fuel_consumption_rate != 0 and max_payload_lift_drag_ratio != 0 and max_payload_final_weight != 0 and test_payload_initial_weight != 0 and test_payload_final_weight != 0 and max_payload_initial_weight != 0 and test_payload_initial_weight != test_payload_final_weight):  
    # Avoid log(0)
    test_payload_total_fuel_consumed = (test_payload_fuel_consumption_rate / max_payload_fuel_consumption_rate)*(test_payload_lift_drag_ratio / max_payload_lift_drag_ratio)*(math.log(max_payload_initial_weight / max_payload_final_weight) /math.log(test_payload_initial_weight / test_payload_final_weight))*max_payload_total_fuel_consumed

# show test_payload_total_fuel_consumed
st.write(f'###### Fuel Consumed Under Max Payload Condition : {max_payload_total_fuel_consumed:.3f}')
st.write(f'###### Fuel Consumed Under Test Payload Condition : {test_payload_total_fuel_consumed:.3f}')

if max_payload_total_fuel_consumed != 0:
    fuel_costing_factor= test_payload_total_fuel_consumed/max_payload_total_fuel_consumed
else:
    fuel_costing_factor= 0.0

Calculating fuel costing factor

In [ ]:
# show fuel costing factor    
st.write(f'##### Fuel Costing Factor : {fuel_costing_factor:.3f}')

st.markdown('-----')

aircraft_turbine_fuel_cost = st.number_input('###### Cost of Aircraft Turbine Fuel (in $/lb)', value = 0.0)
st.write(f'###### Flight Distance : {distance} NM')

# calculating profit
cost_difference = aircraft_turbine_fuel_cost*(max_payload_total_fuel_consumed-test_payload_total_fuel_consumed)
if distance != 0:
    profit_per_NM = cost_difference/distance
else:
    profit_per_NM = 0.0

Displaying the calculated value of test payload total fuel consumed.

In [ ]:
st.write(f'###### Profit per NM : ${profit_per_NM:.2f}')
st.write(f'##### Total Profit : ${cost_difference:.2f}')
st.markdown('----')

Visualization of the test payload total fuel consumed and maximum payload fuel consumed plots.

In [ ]:
# selecting datapoints for plotting
range = [0, distance]
max_payload_costing  = [0, aircraft_turbine_fuel_cost*max_payload_total_fuel_consumed]
test_payload_costing = [0, aircraft_turbine_fuel_cost*test_payload_total_fuel_consumed]

# plotting the data
plt.figure(figsize=(10,6))
plt.plot(range, max_payload_costing, label= 'Cost at Max. Payload Condition')
plt.plot(range, test_payload_costing, label= 'Cost at Test Payload Condition')
plt.fill_between(range, max_payload_costing, test_payload_costing, color= 'green', alpha= 0.3)
plt.title('Cost Difference Between Maximum and Test Payload Condition')
plt.xlabel('Distance')
plt.ylabel('Cost')
plt.legend()
st.pyplot(plt)

## Summary

To summarize it all, we had to find a costing factor by which the operating cost will drop by changing the different variables during a flight. For this, we took the help of Breguet's Range Equation and derived a formula for the same. 

We assigned the factor to a variable $\gamma$, which lies between the range 0 to 1.

Upon finding it, we hardcoded the mathematical function and deployed it to a webserver using streamlit.